In [15]:
import numpy as np
import pandas as pd
df=pd.read_csv('data/train.csv',sep="\t")
df = df.drop('Unnamed: 0', 1)
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,104018,5485,less dens plot,2
1,38706,1844,aspir,2
2,44785,2171,never flag,2
3,98230,5148,elud madonna,2
4,77780,3999,enjoy mindless action movi,2


Replace empty phrase rows to NaN then delete.

In [16]:
#print("size of data with empty phrase rows=",len(df.index))

#define a map to change empty phrases to NaN
def change_nan(x):
    if pd.isnull(x):
        return np.nan 
    if len(x) == 0:
        return np.nan
    return x

df['Phrase'] = df.Phrase.map(change_nan)

#remove NaN rows of df
df=df.dropna()

#reset indexing
df=df.reset_index(drop=True)

#print("size of data after empty phrase rows deleted=",len(df.index))

size of data with empty phrase rows= 124848
size of data after empty phrase rows deleted= 124207


Afer clean, the same same phrases with differen sentiment scores created as follows.

In [31]:
for x in df.Phrase:
    same=df.Phrase[df.Phrase==x]
    if len(same.index)>1:
        break
df.loc[same.index[0]]

KeyboardInterrupt: 

In [ ]:
df.loc[same.index[1]]

Take a phrase as referance to the "phrase" list. Find the same ones in data and record the indexes to the "same" list. In such same phrase groups assign the first one as referance by changing its sentiment score to group average and change other phrases to NaN to remove later from the list. Before doing that sentiment scores should be converted into float data type from intiger(as a default).

In [27]:
#print("data type of sentiment scores= ",df.Sentiment.dtype)

#convert sentiment scores to float to write averaged ones
df.Sentiment=df.Sentiment.astype(float)
df.dtypes
#print("data type of sentiment scores after conversion= ",df.Sentiment.dtype)

#write size of data before merge
#print("data size before merge= ",len(df.index))

#"phrase" holds the current phrase to compare through whole set and find the indexes of same other phrase rows 
#which will written in "same"
#then they will changed to NaN for further clean process
#summation holds the summation of same phrase sentiment scores that will avaraged and written to the first phrase among all same phrases
same=[]
summation=0
phrase=[]

for index,row in df.iterrows():
    if row.Phrase!=np.nan :
        same=df.Phrase[df.Phrase==row.Phrase]
        if len(same.index)>1:
            summation=0
            phrase=df.loc[same.index[0],'Phrase']
            for x in range(1,len(same.index)):
                df.loc[same.index[x],'Phrase']=np.nan
                summation+=df.loc[same.index[x],'Sentiment']
            summation+=df.loc[same.index[0],'Sentiment']
            df.loc[same.index[0],'Sentiment']=summation/len(same.index)

#clean all NaN rows
df=df.dropna()

#write size of data after merge
#print("data size after merge= ",len(df.index))

data type of sentiment scores=  int64
data type of sentiment scores after conversion=  float64
data size before merge=  124207
data size after merge=  75657


Remove NaN assigneds and reindex data.

In [29]:
df=df.dropna()
df=df.reset_index(drop=True)
df.head(15)

,PhraseId,SentenceId,Phrase,Sentiment
0,104018,5485,less dens plot,1.666667
1,38706,1844,aspir,2.142857
2,44785,2171,never flag,2.000000
3,98230,5148,elud madonna,2.000000
4,77780,3999,enjoy mindless action movi,2.000000
5,92189,4798,help screenplay lrb profici singularli cursori...,3.000000
6,152143,8300,disturb world delic ecolog balanc,2.000000
7,150979,8230,unexplain baboon cameo,1.500000
8,55094,2747,unspeak unbear dull featur ream flatli deliv d...,0.000000
9,151451,8259,total loss,0.666667


This cleaned data is sort of vocabulary list. Save file as train-clean.csv under data folder.

In [30]:
df.to_csv("data/train-clean.csv", sep='\t')
df.describe()

,PhraseId,SentenceId,Sentiment
count,75657.000000,75657.000000,75657.000000
mean,79324.027836,4150.265805,2.063290
std,44739.678914,2488.595072,0.877785
min,2.000000,1.000000,0.000000
25%,40892.000000,1960.000000,1.666667
50%,79694.000000,4103.000000,2.000000
75%,118190.000000,6313.000000,2.666667
max,156060.000000,8544.000000,4.000000


In [ ]:
print("done succesfully")